In [1]:
from models import HN, Autoencoder, BASEModel
from collections import OrderedDict, defaultdict
from utils import get_default_device, set_seed, f1_loss
from node import Clients

import os
from tqdm import trange
import pandas as pd
import random

import torch
from torch import nn
import torch.utils.data
from torchvision import transforms

In [2]:
class parameters:
    def __init__(self):
        self.seed = 0
        self.labels_list = ['JOG', 'JUM', 'STD', 'WAL']  # list of activities
        self.outputdim = len(self.labels_list)
        self.data_address = os.path.abspath(os.path.join(
            os.getcwd(), os.pardir)) + "/Datasets/MobiNpy_4_Act/"  # data adress
        self.trial_number = 0  # which trial we use for this test
        self.label_ratio = 0.10  # ratio of labeled data
        self.eval_ratio = 0.30  # ratio of eval data
        self.number_of_client = 1  # total number of clients
        self.server_ID = [0]  # server ID
        self.batch_size = 128  # training batch size
        self.window_size = 30  # window size (for our case 30)
        self.width = 9  # data dimension (AX, AY, AZ) (GX, GY, GZ) (MX, MY, MZ)
        self.n_kernels = 16  # number of kernels for hypernetwork
        self.total_number_of_clients = 59  # total number of subjects (client + server)
        self.learning_rate = 1e-3  # learning rate for optimizer
        self.steps = 10  # total number of epochs
        self.inner_step_for_AE = 20  # number of epochs to fine tunne the Autoencoder
        self.inner_step_server_finetune = 20  # number of steps in the server side to finetune
        self.inner_step_for_model = 20  # number of steps that server fine tune its hn and user embedding parameters
        self.model_loop = False  # feedback loop for user model
        self.inner_step_for_client = 20  # number of steps that user fine tune its model
        self.inner_lr = 1e-3  # user learning rate
        self.inner_wd = 5e-5  # weight decay
        self.inout_channels = 1  # number of channels
        self.hidden = 16  # Autoencoder layer 2 parameters
        self.n_kernels_enc = 3  # autoencoder encoder kernel size
        self.hidden_dim_for_HN = 100  # hidden dimension for hypernetwork
        self.n_kernels_dec = 3  # autoencoder decoder kernel size
        self.latent_rep = 1  # latent reperesentation size
        self.n_hidden_HN = 1  # number of hidden layers in hypernetworks
        self.stride_value = 1  # stride value for autoencoder
        self.padding_value = 1  # padding value for autoencoder
        self.model_hidden_layer = 128  # final model hidden layer size
        self.spec_norm = False  # True if you want to use spectral norm


In [3]:
params = parameters()
set_seed(params.seed)
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
device = get_default_device()

# laoding data
nodes = Clients(address=params.data_address,
                    trial_number=params.trial_number,
                    label_ratio=params.label_ratio,
                    server_ID=params.server_ID,
                    eval_ratio=params.eval_ratio,
                    window_size=params.window_size,
                    width=params.width,
                    transform=transform,
                    num_user=params.total_number_of_clients)


In [4]:
# dataloaders
client_loader = []
client_labeled_loaders = []
eval_loader = []
server_loaders = torch.utils.data.DataLoader(nodes.server_loaders, batch_size=params.batch_size, shuffle=True)

for i in range(params.number_of_client):
    client_loader.append(torch.utils.data.DataLoader(nodes.client_loaders[i], batch_size=params.batch_size, shuffle=True))
    client_labeled_loaders.append(torch.utils.data.DataLoader(nodes.client_labeled_loaders[i], batch_size=params.batch_size, shuffle=True))
    eval_loader.append(torch.utils.data.DataLoader(nodes.eval_data[i], batch_size=params.batch_size, shuffle=True))


In [5]:
AE = Autoencoder(inout_channels=params.inout_channels, hidden=params.hidden, n_kernels_enc=params.n_kernels_enc,
                     n_kernels_dec=params.n_kernels_dec, latent_rep=params.latent_rep, stride_value=params.stride_value,
                     padding_value=params.padding_value)  # initializing the autoencoder
model = BASEModel(latent_rep=params.width * params.window_size * params.latent_rep,
                      out_dim=params.outputdim, hidden_layer=params.model_hidden_layer)  # initilizing the base model

# send models to device

AE.to(device)
model.to(device)



BASEModel(
  (fc1): Linear(in_features=270, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=4, bias=True)
)

In [6]:
# init inner optimizer
inner_optim = torch.optim.Adam(AE.parameters(), lr=params.inner_lr, weight_decay=params.inner_wd)
criteria_AE = torch.nn.BCEWithLogitsLoss()
criteria_model = torch.nn.NLLLoss()



In [7]:
AE.train()
for i in range(params.inner_step_server_finetune):
    inner_optim.zero_grad()
    for sensor_values, _ in client_loader[0]:
        predicted_sensor_values = AE(sensor_values.to(device).float())
        loss = criteria_AE(sensor_values.to(device).float(), predicted_sensor_values.to(device))
        loss.backward()
        inner_optim.step()



In [8]:
with torch.no_grad():
    AE.eval()
    prvs_loss_for_AE_updated = 0
    for sensor_values, _ in eval_loader[0]:
        predicted_sensor_values = AE(sensor_values.to(device).float())
        prvs_loss_for_AE_updated += criteria_AE(sensor_values.to(device).float(),
                                                predicted_sensor_values.to(device)).item() * sensor_values.size(0)
    prvs_loss_for_AE_updated /= len(eval_loader[0].dataset)
    AE.train()

print(prvs_loss_for_AE_updated)


3.905244053765567


In [14]:
model.train()

for i in range(params.inner_step_for_model):
    inner_optim.zero_grad()
    for sensor_values, activity in client_labeled_loaders[0]:
        encoded_sensor_values = AE.encoder(sensor_values.to(device).float())
        predicted_activity = model(encoded_sensor_values)
        loss = criteria_model(predicted_activity.to(device), activity.to(device))
        loss.backward()
        inner_optim.step()




In [15]:
with torch.no_grad():
    model.eval()
    prvs_loss_server_model = 0
    f1_score_server = 0
    for sensor_values, activity in eval_loader[0]:
        #encoded_sensor_values = AE.encoder(sensor_values.to(device).float())
        predicted_activity = model(sensor_values.to(device).float())
        prvs_loss_server_model += criteria_model(
            predicted_activity, activity.to(device)).item() * sensor_values.size(0)
        f1_score_server += f1_loss(activity, predicted_activity) * sensor_values.size(0)
    model.train()
    prvs_loss_server_model /= len(eval_loader[0].dataset)
    f1_score_server /= len(eval_loader[0].dataset)


In [16]:
print("loss: " + str(prvs_loss_server_model))
print("f1: " + str(f1_score_server))


loss: 26.980321292487943
f1: 0.10013285586776272
